In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import cv2

print("open cv2 version: %s" % cv2.__version__)

open cv2 version: 3.3.1


In [2]:
def load_imgs(vidPath, firstFrame, lastFrame):
    '''
    Reads frames from video and stores as a list
    
    Parameters: 
    vidPath (string): Path to video
    
    Returns: 
    list of images
    '''
    
    cap = cv2.VideoCapture(vidPath)
    length = np.arange(start, stop)
    print(len(length)) 
    imgs = []
    for ff in length:
        cap.set(1,ff)
        ret, frame = cap.read()
        if np.mod(ff, 100) == 0:
            print(ff) # prints progress in 50 frames

            # convert to grey
            #img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        imgs.append(frame)
#         print("length of imgs: %s" % len(imgs))
            
    cap.release()
    return imgs

In [3]:
# adjust gamma, if your vid is too dark
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [4]:
def plottracks(ax, x_range, y, frameNum, prob_frame, trackColor):
    ax.plot(x_range, y, color = trackColor)
#     ax.plot(frameNum*1/100, prob_frame, 'o',
#                 color = 'k')
#     ax.set_ylim(0,5)
    ax.axvline(x = frameNum/100, 
              color = 'k',
              alpha = 0.5)
    ax.set_xlabel('Time (second)')
#     ax.set_ylabel('Relative Radial Position')

In [5]:
# blue_hue = {'top_color': [158, 202, 225], 'bottom_color': [8,81, 156]}
# green_hue = {'top_color': [223, 238, 218], 'bottom_color': [0, 82, 33]}
# multiPink = {'top_color': [201, 148, 199], 'bottom_color': [152, 0, 67]}
# orange = {'top_color': [254, 237, 222], 'bottom_color': [166, 54, 3]}

# color_list = [[152, 0, 67], [166, 54, 3], [8,81, 156]]
# from matplotlib import cm

In [6]:
#give path to videofiles
vidpath = r"G:\My Drive\Mahad\Light Level Experiments\Videos Used for Analysis/"
export_path = r'./dataFolder/Output/Proboscis/TrackedVideos/'

# the Ref provides the start and stop frames
path_frames = glob.glob(r"./dataFolders/Output/Step5_v3/" + '*.csv')

In [7]:
visitnum = ['FirstVisit/', 'Lastvisit/']

for visit in visitnum:
    # get the tracks
    direc = r"./dataFolders/Output/Proboscis/FilteredTracks/" + visit
    AllTracks = glob.glob(direc + "*.csv")
    
    for t in AllTracks:
        a, b, c, d = os.path.basename(t).split('_')
        name = a + '_' + b + '_' + c
        print(name)
        
        # get the proboscis track
        track = pd.read_csv(t)
        track.head()

        tracks = pd.DataFrame({'x':track.x,
                                  'y':track.y})

        # get the video section
        videoPath = vidpath + name + '.mp4'
        frame_ref = [f for f in path_frames if name+ '_' in f][0]
        file = pd.read_csv(frame_ref)
        
        if visit == visitnum[0]:
            row = 0
        else:
            row = -1
        
        select = file.iloc[row, :]
        start = int(select['In_Frame'] + 1)
        stop = int(select['Out_Frame'])
        
        
        # get vid info
        cap = cv2.VideoCapture(videoPath)

        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)

        print(length, width, height, fps)
        # load images
        imList = load_imgs(videoPath, start, stop)
        cap.release()

        print("length of section you want to read: %s" % int(stop - start))
        print("length of the image list: %s" % len(imList))
        print("length of the probsocis track: %s" % len(tracks))
        
#         lagPoints = 5 # number of points to plot on each frame
#         alphaVals = np.flip(np.linspace(0, 0.2, lagPoints), axis = 0)
        vidLen = len(tracks)
        
        # make directory to store images
        tempImgDirectory = os.path.join(os.path.dirname(r"./dataFolders/"), "tmpImgs")
        if not os.path.exists(tempImgDirectory):
            os.mkdir(tempImgDirectory)
            
            
        #######################################################
        outputImgs = []
        for frameNum in np.arange(0, vidLen):
            
            # adjust gamma
            image = adjust_gamma(imList[frameNum], 1.5)
            tt = tracks.loc[frameNum, :]
            x = tt.x 
            y = tt.y
            
            output = image.copy()
            rad_circle = 5
            color = (0, 255, 0)
            if np.isnan(x) or np.isnan(y):
                continue
            else:
                cv2.circle(output, (int(x),int(y)),
                           rad_circle, 
                           color,
                           -1)
            
            # save img
            cv2.imwrite(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"),
                       output)
#             plt.savefig(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"))
#             plt.close()
            
            if np.mod(frameNum, 100) == 0:
                print(frameNum, " of ", vidLen)
                
        ##################################################################################
        
        # convert images to video with ffmpeg
        os.chdir(tempImgDirectory)
        print(tempImgDirectory)

        # use ffmpeg to convert directory to video
        # -r is output frame rate
        os.system('ffmpeg -start_number 0 -r 20 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y outputVid.mp4')

        nameofVideo = r'./' + name + '_' + visit[:-1] + '.mp4'
        os.rename(r'./outputVid.mp4', nameofVideo)
        os.chdir('../../')

        # delete images from directory
        delFiles = [f for f in os.listdir(tempImgDirectory) if f.endswith("png")]
        dlfs = [os.remove(os.path.join(tempImgDirectory, delFiles[ii])) for ii in range(len(delFiles))]

L0.1_c-3_m10
140688 648 488 30.0
335
20400
20500
20600
20700
length of section you want to read: 335
length of the image list: 335
length of the probsocis track: 273
0  of  273
100  of  273
200  of  273
./dataFolders\tmpImgs


FileExistsError: [WinError 183] Cannot create a file when that file already exists: './outputVid.mp4' -> './L0.1_c-3_m10_FirstVisit.mp4'

In [ ]:
os.chdir('../../')
os.getcwd()